In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:19pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:16pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))


<b><font size="7" color="red">ch14. 웹데이터 수집</font></b>
# 1절. BEAUTIFULSOUP과 parser
'pip install bs4' 아나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함

- 공식 Documentation : https://beautiful-soup-4.readthedocs.io/en/latest/

In [2]:
# 내 Local PC의 파일을 url(http://~)처럼 접근
import requests # http 요청 처리하는 Lib
from requests_file import FileAdapter # file://프로토콜을 다루기 위함

In [3]:
s = requests.Session() # HTTP 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
# file://경로로 들어오면 이 요청은 c:(로컬파일)을 접근
response = s.get('file:///ai/ch14_sample.html')
response

<Response [200]>

In [4]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다')
    

잘 접근하였습니다.


In [5]:
response.status_code
    # 200 : 정상
    # 404 : 없는페이지
    # 406 : get, post 오류

200

In [6]:
response.content # 바이너리 형식의 html 

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [7]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [8]:
response.text # HTML파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [9]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, #response.text
                    "html.parser")
# soup

In [10]:
# soup.select_one('선택자') : 해당선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성들 :', el.attrs)
print('el의 id속성 :', el.attrs['id']) # el.attrs은 딕셔너리
print('el의 id속성 :', el.attrs.get('id'))
print('el의 href속성 :', el.attrs.get('href'))
print('el의 태그이름 :', el.name)

el : <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text : Hello, CSS
el.string : Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text'}
el의 id속성 : text
el의 id속성 : text
el의 href속성 : None
el의 태그이름 : h1


In [11]:
# soup.select('선택자') : 해당 선택자 모두
els = soup.select('h1') # h1태그를 list
print('els :', els)
print('els의 text들 :', [el.text for el in els])
print('els의 string들 :', [el.string for el in els])
print('els의 속성들 :', [el.attrs for el in els])
print('els의 class 속성들 :', [el.attrs.get('class') for el in els])

els : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들 : ['Hello, CSS', 'Hi, CSS']
els의 string들 : ['Hello, CSS', 'Hi, CSS']
els의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]
els의 class 속성들 : [['greeting', 'css'], ['css']]


In [12]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('select_one :', soup.select_one('h1.css') )
print('find :', soup.find('h1', {'class':'css'}) ) # soup.select_one('h1.css')
print('find :', soup.find('h1', class_='css') )
print()
print('select_one :', soup.select_one('h1#text') )
print('find :', soup.find('h1', {'id':'text'}))

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [13]:
# soup.find_all(태그, 속성) : soup.select('선택자')와 유사
print('select :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [14]:
print('select :', soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1', 'span'], {'class':'css'}))
print('find_all :', soup.find_all('h1', class_='css') + \
                     soup.find_all('span'))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [15]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a', {'class','css'}))
print('find(None) :', soup.find('a', {'class':'css'}))


select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 BeautifulSoup을 활용한 html 웹 데이터수집
### 1) 환율정보 가져오기 (네이버증권 -> 시장지표_)
-https://finance.naver.com/marketindex/

In [16]:
# 크롤링 허용범위는 사이트마다 ~/robots.txt에서 확인
    # Allow : / - 사이트의 모든 경로(/)애ㅔ 대한 크롤링 허용

In [17]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
#response.status_code
# response.text / response.content.decode('cp949')
soup = BeautifulSoup(response.text, "html.parser")



In [18]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
# response.read() / responseread().decode('cp949')
soup = BeautifulSoup(response, "html.parser")

In [19]:
# div.head_info > span.value (find함수)
prices = []
headinfos = soup.find_all('div', class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_ = 'value')
    #print(price.text.repalce(',', ''))
    prices.append(float(''.join(price.text.split(','))))
print(prices)

[1464.0, 949.11, 1695.82, 205.57, 153.98, 1.16, 1.318, 99.31, 61.04, 1704.69, 4116.3, 194893.24]


In [20]:
# div.head_info > span.value (select함수)
prices = soup.select('div.head_info > span.value')
print([float(p.text.replace(',','')) for p in prices])

[1464.0, 949.11, 1695.82, 205.57, 153.98, 1.16, 1.318, 99.31, 61.04, 1704.69, 4116.3, 194893.24]


In [21]:
titles = soup.select('h3.h_lst > span.blind')
for t in titles:
    print(t.text, end='\t')

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [22]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7, '') # 7번째에 '' 추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [23]:
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')

상승	상승	상승	상승	하락	상승	상승	하락	상승	상승	하락	상승	

In [24]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [25]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text,
                                 prices[idx].text,
                                 units[idx],
                                 statuses[idx].text))

미국 USD : 1,464.00원 - 상승
일본 JPY(100엔) : 949.11원 - 상승
유럽연합 EUR : 1,695.82원 - 상승
중국 CNY : 205.57원 - 상승
달러/일본 엔 : 153.9800엔 - 하락
유로/달러 : 1.1600달러 - 상승
영국 파운드/달러 : 1.3180달러 - 상승
달러인덱스 : 99.3100 - 하락
WTI : 61.04달러 - 상승
휘발유 : 1704.69원 - 상승
국제 금 : 4116.3달러 - 하락
국내 금 : 194893.24원 - 상승


In [26]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print("{} : {}{} -{}".format(title.text,
                                price.text,
                                unit,
                                status.text))

미국 USD : 1,464.00원 -상승
일본 JPY(100엔) : 949.11원 -상승
유럽연합 EUR : 1,695.82원 -상승
중국 CNY : 205.57원 -상승
달러/일본 엔 : 153.9800엔 -하락
유로/달러 : 1.1600달러 -상승
영국 파운드/달러 : 1.3180달러 -상승
달러인덱스 : 99.3100 -하락
WTI : 61.04달러 -상승
휘발유 : 1704.69원 -상승
국제 금 : 4116.3달러 -하락
국내 금 : 194893.24원 -상승


### 2) 이번주 로또 번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin (google에 로또번호 당첨번호 검색)
````
11997회(2025년 11월 08일 추첨)
당첨번호 [1,5,7,26,28,43]
보너스 30
````


In [27]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
#soup

In [28]:
# 방법2
from urllib.request import urlopen
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
response.status
soup = BeautifulSoup(response, "html.parser")
# soup

In [29]:
times = soup.select_one('div.win_result strong').text
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win  span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [30]:
print(times, date)
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

1197회 (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


### 3) 다음 검색 리스트
```
no title   href
0 "한풀 꺾인 비트코인 https://daum.net/v/202511110094711892
```

In [31]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

items_find_list = [] #검색한 결과를 담을 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    #print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                           'title':item.text,
                           'link':item.attrs.get('href')})
import pandas as pd

pd.DataFrame(items_find_list)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,가상자산 일제히 급락… 비트코인 10만달러 위태,http://v.daum.net/v/20251112080939100
1,1,큰손들은 털고 나간다…비트코인 10만 달러 붕괴 조짐,http://v.daum.net/v/20251112082149370
2,2,"‘비트코인 9.6조 압수’ 中 사기범, 영국서 징역 11년 8개월형",http://v.daum.net/v/20251112093346011
3,3,"영국서 초호화생활 중국인 여성 중형, 왜?…다단계 사기로 ‘비트코인 10조’ 세탁",http://v.daum.net/v/20251112093904308
4,4,압수 비트코인만 9조원···사기친 돈 세탁한 중국인 영국서 징역 11년,http://v.daum.net/v/20251112094247439
5,5,"""비트코인 9조 최대 압수""...'중국 가상화폐 여왕' 영국서 징역 11년",http://v.daum.net/v/20251112064009731
6,6,비트코인 9조 4천억 압수...자금세탁 중국인 징역11년8개월,http://v.daum.net/v/20251112073602477
7,7,“새로운 기부라고 생각” 서울대병원에 1비트코인 기부한 70대…가격 보니,http://v.daum.net/v/20251112064050739
8,8,"꿈틀대던 비트코인, 다시 10.5만달러대…'고래 매도' 조정 신호?",http://v.daum.net/v/20251111161648648
9,9,"70대 개인투자자, 서울대병원에 '1비트코인' 기부…""새로운 형태 나눔 문화 확산...",http://v.daum.net/v/20251112090849904


In [32]:
# 방법2 : 불가
from urllib.request import urlopen
from urllib.parse import quote
word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 변환
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = urlopen(url)

import pandas as pd



https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


In [33]:
items_find_list = [] #검색한 결과를 담을 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs['href']])
pd.DataFrame(items_find_list, columns=['번호','제목','링크'])

,번호,제목,링크
0,0,가상자산 일제히 급락… 비트코인 10만달러 위태,http://v.daum.net/v/20251112080939100
1,1,큰손들은 털고 나간다…비트코인 10만 달러 붕괴 조짐,http://v.daum.net/v/20251112082149370
2,2,"‘비트코인 9.6조 압수’ 中 사기범, 영국서 징역 11년 8개월형",http://v.daum.net/v/20251112093346011
3,3,"영국서 초호화생활 중국인 여성 중형, 왜?…다단계 사기로 ‘비트코인 10조’ 세탁",http://v.daum.net/v/20251112093904308
4,4,압수 비트코인만 9조원···사기친 돈 세탁한 중국인 영국서 징역 11년,http://v.daum.net/v/20251112094247439
5,5,"""비트코인 9조 최대 압수""...'중국 가상화폐 여왕' 영국서 징역 11년",http://v.daum.net/v/20251112064009731
6,6,비트코인 9조 4천억 압수...자금세탁 중국인 징역11년8개월,http://v.daum.net/v/20251112073602477
7,7,“새로운 기부라고 생각” 서울대병원에 1비트코인 기부한 70대…가격 보니,http://v.daum.net/v/20251112064050739
8,8,"꿈틀대던 비트코인, 다시 10.5만달러대…'고래 매도' 조정 신호?",http://v.daum.net/v/20251111161648648
9,9,"70대 개인투자자, 서울대병원에 '1비트코인' 기부…""새로운 형태 나눔 문화 확산...",http://v.daum.net/v/20251112090849904


In [34]:

items_find_list = [] # 검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    #items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link':a.attrs['href'],
    })
pd.DataFrame(items_find_list)

,no,title,link
0,0,가상자산 일제히 급락… 비트코인 10만달러 위태,http://v.daum.net/v/20251112080939100
1,1,큰손들은 털고 나간다…비트코인 10만 달러 붕괴 조짐,http://v.daum.net/v/20251112082149370
2,2,"‘비트코인 9.6조 압수’ 中 사기범, 영국서 징역 11년 8개월형",http://v.daum.net/v/20251112093346011
3,3,"영국서 초호화생활 중국인 여성 중형, 왜?…다단계 사기로 ‘비트코인 10조’ 세탁",http://v.daum.net/v/20251112093904308
4,4,압수 비트코인만 9조원···사기친 돈 세탁한 중국인 영국서 징역 11년,http://v.daum.net/v/20251112094247439
5,5,"""비트코인 9조 최대 압수""...'중국 가상화폐 여왕' 영국서 징역 11년",http://v.daum.net/v/20251112064009731
6,6,비트코인 9조 4천억 압수...자금세탁 중국인 징역11년8개월,http://v.daum.net/v/20251112073602477
7,7,“새로운 기부라고 생각” 서울대병원에 1비트코인 기부한 70대…가격 보니,http://v.daum.net/v/20251112064050739
8,8,"꿈틀대던 비트코인, 다시 10.5만달러대…'고래 매도' 조정 신호?",http://v.daum.net/v/20251111161648648
9,9,"70대 개인투자자, 서울대병원에 '1비트코인' 기부…""새로운 형태 나눔 문화 확산...",http://v.daum.net/v/20251112090849904


In [35]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    items_find_list = [] # 검색한 결과를 담는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    print(keyword)
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [36]:
collect_news_list('비트코인', 3)

비트코인


[[20,
  ' 파생상품 대거 청산, 암호화폐 일제 하락…리플 5%↓(상보) ',
  'http://v.daum.net/v/20251112073111389'],
 [21,
  ' 투자 사기 후 영국으로 도주한 중국인… 법원 "비트코인 9조원 압수" ',
  'http://v.daum.net/v/20251112083740757'],
 [22,
  ' 코스피 4097.44에 하락 출발…환율 1461원 ',
  'http://v.daum.net/v/20251112090609808'],
 [23,
  ' 비트코인, 美 정부 셧다운 해제 관망에 10만6000달러선 [크립토브리핑] ',
  'http://v.daum.net/v/20251111103648808'],
 [24,
  ' 개미 투자자 ‘데이터 무장’ 돕는다…업비트, 데이터랩 ‘인사이트’ 기능 강화 ',
  'http://v.daum.net/v/20251112090600801'],
 [25,
  " 9조 비트코인 들고 런던으로 튄 '中 폰지의 여왕', 징역 11년8개월 ",
  'http://v.daum.net/v/20251112062715613'],
 [26,
  ' 美 ‘셧다운 해제 기대감’에 가상자산 오름세… 비트코인 10만6200달러 ',
  'http://v.daum.net/v/20251111083345066'],
 [27,
  ' 12만명 속여 ‘비트코인 9조’ 40대 중국여성…도망간 英서 징역 11년8개월 ',
  'http://v.daum.net/v/20251112064227765'],
 [28,
  ' "내년에도 비트코인·K-방산"…\'지정학적 위기\' 뜨는 자산 ',
  'http://v.daum.net/v/20251110095715363'],
 [29,
  ' 서울대병원, 첫 비트코인 기부 받았다…1억5700만원 병원발전기금으로 ',
  'http://v.daum.net/v/20251111170538874']]

In [37]:
import time
keywords = ['청바지','동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결괄르 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번쨰 검색어 {keyword} 검색 결과 수집 중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword,page))
        elif i == 1:
            result1.extend(collect_news_list(keyword,page))
        time.sleep(0.5)
    
        

===1번쨰 검색어 청바지 검색 결과 수집 중입니다===
청바지
청바지
청바지
===2번쨰 검색어 동대문 검색 결과 수집 중입니다===
동대문
동대문
동대문


In [38]:
result0_df = pd.DataFrame(result0, columns=['no','title','link'])
result1_df = pd.DataFrame(result1, columns=['no','title','link'])
result1_df.head()

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 ‘핫플’ 됐다",http://v.daum.net/v/20251111111251709
1,1,"동대문구, 사회복지시설 종사자 독감 예방접종 비용 지원",http://v.daum.net/v/20251112081345203
2,2,"동대문구, ‘2025 아이돌봄서비스 평가’ A등급(우수) 선정",http://v.daum.net/v/20251111172914828
3,3,"노보텔 앰배서더 동대문, 연말 맞아 '페스티브 파티’ 패키지 출시",http://v.daum.net/v/20251111155426418
4,4,"동대문구, 11월 학부모·학생 맞춤형 교육 특강",http://v.daum.net/v/20251111151146411


In [39]:
result0_df.to_csv('data/ch14_{keywords{0}}.csv',index=False, encoding='cp949')
result1_df.to_csv('data/ch14_{keywords{1}}.csv',index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라어저를 통해 요청하는 것 처럼 보이기

In [40]:
# 방법2 :
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인') # 한글을 encoding 처리
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
# headers = {'User-Agent':
#           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request = Request(url, headers = headers)
request = Request(url)
request.add_header('User-Agent',
                  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36')
response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
# soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


###4)

In [41]:
# 방법 1
import requests
url = 'https://www.melon.com/chart/hot100/index.htm'
melonpage = requests.get(url)
melonpage
soup = BeautifulSoup(melonpage.text, 'html.parser')
soup




In [42]:
# 방법2
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/hot100/index.htm'
# melonpage = urlopen(url) # 에러남
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage, 'html.parser')
#soup

In [43]:
# 방법1 : User-Agent 추가
url = 'https://www.melon.com/chart/hot100/index.htm'

headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}

melonpage  = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melponpage.content
                    'html.parser')
ranks = soup.select('td div.wrap.t_center > span.rank')

titles = soup.select('div.ellipsis.rank01 > span')

singers = soup.select('div.ellipsis.rank02 > span')

#[singer.text.strip()[:20] for singer in singers]

for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXX
2위 | 타임캡슐 - 다비치
3위 | 달리 표현할 수 없어요 - 로이킴
4위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
5위 | Good Goodbye - 화사 (HWASA)
6위 | IRIS OUT - Kenshi Yonezu
7위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)
8위 | Hollywood Action - BOYNEXTDOOR
9위 | XOXZ - IVE (아이브)
10위 | 한번 더 이별 - 이창섭
11위 | Rich Man - aespa
12위 | body - 다영 (DAYOUNG)
13위 | FOCUS - Hearts2Hearts (하츠투하츠
14위 | BBUU! - PLAVE
15위 | 순간을 영원처럼 - 임영웅
16위 | 봉숭아 - PLAVE
17위 | OVERDRIVE - TWS (투어스)
18위 | 들꽃이 될게요 - 임영웅
19위 | JANE DOE - Kenshi Yonezu, Hikar
20위 | 그댈 위한 멜로디 - 임영웅
21위 | 숨바꼭질 (Hide and Seek) - PLAVE
22위 | 비가 와서 - 임영웅
23위 | 가만히 눈을 감고 - DK(디셈버)
24위 | 답장을 보낸지 - 임영웅
25위 | 나였으면 - 나윤권, 도경수(D.O.)
26위 | 알겠어요 미안해요 - 임영웅
27위 | ULSSIGU - 임영웅
28위 | 바이, 썸머 - 아이유
29위 | Wonderful Life - 임영웅
30위 | 꿈의 버스 - DAY6 (데이식스)
31위 | 숨바꼭질 (Hide and Seek) - PLAVE
32위 | 돌아보지 마세요 - 임영웅
33위 | 나는야 HERO - 임영웅
34위 | 우리에게 안녕 - 임영웅
35위 | 당신을 위하여 - 이보람 (씨야)
36위 | 있잖아 - BOYNEXTDOOR
37위 | Be My Light - 이세계아이돌
38위 | 0+0 - 한로로
39위 | Nameless - 이

### 5) 네이버 지식인으로 검색(open API 사용X)
- 특정 keyword를 특정페이지 수만큼

In [44]:

# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '쳇지피티'
# url = 'https://kin.naver.com/search/list.naver'
# params = {'query' : keyword}
# response = get(url, params=params)
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
response = get(url)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')
# soup


200


In [45]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
keyword = quote('쳇지피티')
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
print(url)
response = urlopen(url)
print(response.status)
soup = BeautifulSoup(response, 'html.parser')
#soup

https://kin.naver.com/search/list.naver?query=%EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0
200


In [46]:

# 페이징 포함
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('쳇지피티')
pages = 3
items_list = [] # 크롤링한 데이터를 담을 list(title, link)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = urlopen(url)
    # print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    items = soup.select('dt > a')
    for item in items:
        title = item.text
        link  = item.attrs.get('href')
        items_list.append({
            'title': title,
            'link' : link
        })
df = pd.DataFrame(items_list)
print(df.shape)
print(df.loc[29, 'link'])
df.sample()

(30, 2)
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10303&docId=484943564&qb=7LOH7KeA7ZS87Yuw&enc=utf8


,title,link
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...


## 2.2 openAPI사용 : json 웹데이터 수집
### 1) 네이버 지식인으로 검색(open API 사용 O)
- 네이버 개발자센터에서 애플리케이션 등록(이름/검색/WEB설정http://localhost)
- .env파일에 CLIEnt_ID/CLIENT_SECRET 환경변수 저장
- 환경변수를 읽기 위해서 `pip install dotenv`
- 특정 keyword를 지식검색(데이터수 30개)

In [47]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [48]:
# 환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv(
    #dotenv_path='.env'
           )# 현 소스와 같은 폴더내의 .env를 메모리에 Load

#print(os.getenv('CLIENT_ID'))
#print(os.getenv('CLIENT_SECRET'))

True

In [49]:
#방법2
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin?query=" + encText # JSON 결과
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
request = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(request)
rescode = response.status
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    print(response_body.decode('utf-8')[:50])
else:
    print("Error Code:" + rescode)
items = json.loads(response_body.decode('utf-8'))['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list, columns=['title', 'link', 'description']).head()

<class 'str'>
{
	"lastBuildDate":"Wed, 12 Nov 2025 09:44:00 +090


,title,link,description
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...,쳇지피티 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요...
1,쳇지피티 질문,https://kin.naver.com/qna/detail.naver?d1id=1&...,제가 쳇지피티 한테 물어볼게 있어서 지피티야 이러면서 운을 뗐는데 애가 갑자기 제 ...
2,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티한테 질문 제대로 하려면 어떻게 해야 하냐요?... 쳇지피티 한테 하는질문도...
3,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지) 남매입니다 남자 1986년 9...
4,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료버전으로 카톡대화창 전송해서 상대의심리에 관한 분석요청을 많이했는데 첨...


In [50]:
# 방법1
import requests
import pandas as pd
import json # response텍스트를 json스타일의 딕셔너리로
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '쳇지피티'
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f'https://openapi.naver.com/v1/search/kin?query={query}&display=30'
# response = requests.get(url, headers=headers)
url = 'https://openapi.naver.com/v1/search/kin'
params = {'query':query, 'display':30}
response = requests.get(url, params=params, headers=headers)
# print(response.text[:500])
# items = json.loads(response.text)['items']
items = response.json()['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list, columns=['title', 'link', 'description']).head()

,title,link,description
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...,쳇지피티 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요...
1,쳇지피티 질문,https://kin.naver.com/qna/detail.naver?d1id=1&...,제가 쳇지피티 한테 물어볼게 있어서 지피티야 이러면서 운을 뗐는데 애가 갑자기 제 ...
2,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티한테 질문 제대로 하려면 어떻게 해야 하냐요?... 쳇지피티 한테 하는질문도...
3,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지) 남매입니다 남자 1986년 9...
4,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료버전으로 카톡대화창 전송해서 상대의심리에 관한 분석요청을 많이했는데 첨...


### quiz) 네이버 open API를 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지.csv출력
```
    제목,링크,썸네일,sizeheight,sizewidth
```

In [51]:
ef get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법2)'
    from urllib.request import urlopen, Request
    from urllib.parse import quote
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    encText = quote(query)
    url = f'https://openapi.naver.com/v1/search/image?query={encText}&display=100'
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    request = Request(url, headers=headers)
    response = urlopen(request)
    # print(response.read().decode('utf-8'))
    items = json.loads(response.read().decode('utf-8'))['items']
    items_list = [] # 정보가 담길 리스트
    for item in enumerate(items):
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        # 이미지 파일 저장
        save_image('메일', idx, link, query)
        save_image('썸네일', idx, thumbnail, query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행완료 = = =')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('image/ch14_청바지.csv')
df.sample()

SyntaxError: invalid syntax (3440439115.py, line 1)

In [ ]:

def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

In [ ]:
print(df.loc[99, '링크'])
print(df.loc[99, '썸네일'])

In [ ]:

def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query.확장자로 local에 저장'
    import requests
    file_extension = link.split('.')[-1] # 확장자
    quote_index = file_extension.find('?') # 확장자뒤에 ?가 있는 위치 .jpg?w=780
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
    img = requests.get(link).content # 바이너리
    with open(f'image/{attr}_{idx+1:02}_{query}.{file_extension}', 'wb') as f:
        f.write(img)
save_image('메인', 0, df.loc[25, '링크'], '청바지')

In [ ]:
def get_image_list_save_image(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = [] # 정보가 담길 리스트
    for idx, item in enumerate(items):
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        # 이미지 파일 저장
        save_image('메인', idx, link, query)
        save_image('썸네일', idx, thumbnail, query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행중 = = =')
    print('= = = 완료 = = =')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('image/ch14_청바지.csv')
df.sample()

## 2.3 XML 웹데이터 수집
- RSS /open API 을 통한 XML 웹데이터 수집

### 1) 전국 날씨 RSS를 BeautifulSoup을 이용한 크롤링



In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
items_list = []
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'
# 방법1
# target = requests.get(url)
# soup = BeautifulSoup(target.text, "xml") # pip install lxml 필요없음
# 방법2
target = urlopen(url)
soup = BeautifulSoup(target, "xml")
local_tas = soup.select('local_ta')
for local_ta in local_tas:
    local_name = local_ta.select_one('local_ta_name').text
    week1_normalYear = local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local_ta.select_one('week1_local_ta_similarRange').text
    week1_minVal = local_ta.select_one('week1_local_ta_minVal').text
    week1_similarVal = local_ta.select_one('week1_local_ta_similarVal').text
    week1_maxVal = local_ta.select_one('week1_local_ta_maxVal').text
    items_list.append({
        '지역':local_name,
        '1주평년기온':week1_normalYear,
        '1주기온범위':week1_similarRange,
        '1주낮을확률':week1_minVal,
        '1주비슷할확률':week1_similarVal,
        '1주높을확률':week1_maxVal
    })
df = pd.DataFrame(items_list)
df['1주평년기온'] = df['1주평년기온'].astype(np.float64)
df['1주낮을확률'] = df['1주낮을확률'].astype(np.int16)
df['1주비슷할확률'] = df['1주비슷할확률'].astype(np.int16)
df['1주높을확률'] = df['1주높을확률'].astype(np.int16)
df.head()


### 2) XML응답하는 open API 활용
- data.go.kr에서
    - 서울특별시_노선정보조회 서비스(버스ID, 정류장목록과정류장ID)
    - 서울특별시_버스위치정보조회 서비스(실시간 버스 위치 목록)
    

In [ ]:
# STEP 1 버스번호의 버스 id받아오기
# 서울특별시_노선정보조회 서비스 - 3번 기능(getBusRouteList) 이용

In [53]:
from dotenv import load_dotenv
import os 
load_dotenv()
# print(os.getenv('KEY1'))


True

In [54]:
import requests
from bs4 import BeautifulSoup
# urlretrive(url, 저장경로) : url의 파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote
#busNum = quote('종로07')
busNum = '162'
key = os.getenv('KEY1')
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
print(url)
savefilename1 = 'data/ch14_busInfo.xml'
urlretrieve(url, savefilename1) #xml파일(url)을 savafilename으로 저장
with open(savefilename1, encoding='utf-8') as f:
    xml = f.read();
soup = BeautifulSoup(xml, 'xml')


http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey=ZS3iEVZ82iG9dvZc3urYq03YK9x0dwPWsaXcaNd1XACv8yWys1TONILRRSHiRNyrFo1qVk379RdHIUhAtRaL5w%3D%3D&strSrch=162


In [55]:
import requests 
from urllib.parse import quote
busNum = '162'
key = os.getenv('KEY1')
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'xml')
for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        print(busRouteId)
        break;
print('busRouteId =', busRouteId)

100100034
busRouteId = 100100034


In [56]:
# stpe2. 버스id로 버스정류장목록받아오기(정류장명, 정류장id, 경도, 위도)
# 서울특별시_노선정보조회 서비스 - 4번기능()

In [63]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
response = requests.get(url2)
soup = BeautifulSoup(response.content,'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수 : {len(itemLists)}')

bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text # 정류소 이름
    station   = itemList.find('station').text # 정류소 id
    gpsX      = itemList.find('gpsX').text # 경도
    gpsY      = itemList.find('gpsY').text # 위도
    bus_station.append([stationNm, station, gpsX, gpsY])
df_station = pd.DataFrame(bus_station, columns=['정류소명','id','경도','위도'])
df_station.head()

162번 정류장 갯수 : 77


,정류소명,id,경도,위도
0,정릉산장아파트,107000071,127.003343,37.616712
1,정릉4동주민센터.경국사,107000073,127.006345,37.613529
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899
3,성북청수도서관.정릉4동성당,107000075,127.0084193769,37.6115696748
4,정릉시장입구,107000077,127.0098212542,37.6084653256


In [58]:
# step3. 버스 id로 실시간위치정보를 받아오기(차량번호, 혼잡도, 최종정류장id,다음정류장id, 도착소요시간)
# 서울특별시_버스위치정보조회 서비스 - 2번(getBusPosByRtidList)


In [66]:
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey={key}&busRouteId={busRouteId}'
# print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text, 'xml')
itemLists = soup.select('itemList')
print(f'운행중인 버스는 {len(itemLists)}대입니다')
bus_position = []
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text # 차량번호
    congetion = itemList.select_one('congetion').text # 혼잡도(congetion)
    # 0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡
    congetion = '없음' if congetion=='0' \
            else '여유' if congetion=='3' \
            else '보통' if congetion=='4' \
            else '혼잡' if congetion=='5' else '매우혼잡'
    gpsX = itemList.select_one('gpsX').text # 경도
    gpsY = itemList.select_one('gpsY').text # 위도
    lastStnId = itemList.select_one('lastStnId').text # 최종정류소id
    nextStId = itemList.select_one('nextStId').text # 다음정류소id
    nextStTm = itemList.select_one('nextStTm').text # 다음정류소까지 소요시간
    
    bus_position.append({
        '차량번호':plainNo,
        '혼잡도':congetion,
        '경도':gpsX,
        '위도':gpsY,
        '최종정류소id':lastStnId,
        '다음정류소id':nextStId,
        '도착소요시간':nextStTm
    })
df_position = pd.DataFrame(bus_position)
df_position.head(2)

운행중인 버스는 22대입니다


,차량번호,혼잡도,경도,위도,최종정류소id,다음정류소id,도착소요시간
0,서울74사2244,여유,127.008532,37.611432,107000075,107000079,232
1,서울74사3381,여유,127.014223,37.598817,107000170,101000042,1858


In [70]:
df_station.loc[df_station['id'] == '107000518', '정류소명'].iloc[0]

'북한산보국문역2번출구'

In [104]:
df_station.loc[df_station['id']=='107000079','정류소명'].iloc[0]

'정릉우체국앞'

In [102]:
def station_name(row):
    row = row.copy()
    'row의 최종정류소명과 다음정류소명을 추가하고, 도착소요시간을 분으로 바꿔 return'
    row['이전정류소명'] = df_station.loc[df_station['id']==row['최종정류소id'],'정류소명'].iloc[0]
    row['다음정류소명'] = df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    minite = int(row['도착소요시간'])//60 # //몫연산자
    sec    = int(row['도착소요시간']) % 60
    row['도착소요시간'] = f'{minite}분{sec}초'
    return row
#station_name(df_position.loc[20])

In [103]:

df = df_position.apply(station_name, axis=1)
df.drop(['최종정류소id','다음정류소id'], axis=1).head()


ValueError: invalid literal for int() with base 10: '3분52초'

# 3절 연습문제(Quiz1)
- yes24의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장하세요.
    * 베스트셀러 정보 수집 주소 : : http://www.yes24.com/24/category/bestseller (1페이지만 추출)
    * https://www.yes24.com/product/category/bestseller?categoryNumber=001&pageNumber (여러 페이지 추출)
    
    ```
    ch14_yes24.csv나 ch14_yes24.txt의 내용(구분문자를 ,나 \t,....)
        -순위\t책이름\t저자및출판사\t가격
        1\ [도서]트렌드코리아2026\t"김난도,전미영,최지혜,권정윤,한다혜 | 미래의창" \t18.000원
        24, [만화]그 비스트 어쩌구, 후쿠다신이치글그림/박연지역 | 소미미디어, 25, 200원
    ```

In [ ]:
url ='https://www.yes24.com/product/category/bestseller?categoryNumber=001&pageNumber=2'

In [46]:
# import
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [47]:
# 방법1
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.replace('\r','').strip() for authors_el in authors_els]
    for i, author in enumerate(authors):
        match = re.search(r"(.*)외 \d+명", author)
        if(match):
            authors[i] = match.group()
            print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

0 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름
10 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름


,순위,책이름,저자,출판사,가격
0,1,트렌드 코리아 2026,"김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 7명",미래의창,"18,000원"


In [ ]:
# 방법2
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = urlopen(url)
    soup = BeautifulSoup(response, 'html.parser')
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.split('정보 더 보기/감추기')[0].strip() 
                                   for authors_el in authors_els]
#     for i, author in enumerate(authors):
#         match = re.search(r"(.*)외 \d+명", author)
#         if(match):
#             authors[i] = match.group()
#             print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

In [60]:
import requests
from bs4 import BeautifulSoup
page = 2
url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
print(url)
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

bestseller_list = [] #검색한 결과를 담을 리스트

                       
ranks_els = soup.select('div.img_upper > em.ico.rank')
ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
titles_els=soup.select("div.info_row > a.gd_name")
titles = [titles_el.text for titles_el in titles_els]
authors_els = soup.select('span.authPub.info_auth')
authors = [authors_el.text.split('정보 더 보기/감추기')[0].strip() 
                               for authors_el in authors_els]
publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
publishers = [publishers_el.text for publishers_el in publishers_els]
prices_els = soup.select("div.info_row > strong.txt_num")
prices = [prices_el.text for prices_el in prices_els]

for idx, rank in enumerate(ranks):
    bestseller_list.append({'순위':rank,'책이름':title,'저자':author,'가격':price})

import pandas as pd

pd.DataFrame(bestseller_list)

https://www.yes24.com/product/category/bestseller?pageNumber=2


,순위,책이름,저자,가격
0,25,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
1,26,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
2,27,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
3,28,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
4,29,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
5,30,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
6,31,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
7,32,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
8,33,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
9,34,너를 미워할 시간에 나를 사랑하기로 했다,윤서진 저,"16,920원"
